### All necessary imports

In [137]:
from lxml.html import parse
from urllib.request import urlopen
from pandas.io.parsers import TextParser

### Storing the cik and acc number in variables

In [138]:
#cik =
#acc =

### Parsing the url 

In [2]:
parsed = parse(urlopen('https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.htm'))
doc = parsed.getroot()

In [3]:
links = doc.findall('.//a')

In [4]:
url = [lnk.get('href')for lnk in doc.findall('.//a')]

In [100]:
url[10:23]

In [6]:
tables = doc.findall('.//table')

In [125]:
example = tables[0]

In [65]:
rows = example.findall('.//tr')

In [66]:
#def _unpack(row, kind='td'):
#    elts = row.findall('.//%s' % kind)
#    return [val.text_content() for val in elts]

#def parse_options_data(table):
#    rows=table.findall('.//tr')
#    header = _unpack(rows[0], kind ='th')
#    data = [_unpack(r) for r in rows[1:]]
#    return TextParser(data, names = header).get_chunk()

#example_data = parse_options_data(example)
#example_data[:10]

#for r in range(len(example_data)):
#        d= example_data['Description'][r]
#        if d =='10-Q' :
#            Doc.append(example_data['Document'][r])



In [67]:
print(_unpack(rows[0], kind="th"))

['Seq', 'Description', 'Document', 'Type', 'Size']


In [129]:
Doc=[]
for t in range(len(tables)):
    example = tables[t]
    
    rows = example.findall('.//tr')
    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content() for val in elts]

    def parse_options_data(table):
        rows=table.findall('.//tr')
        header = _unpack(rows[0], kind ='th')
        data = [_unpack(r) for r in rows[1:]]
        return TextParser(data, names = header).get_chunk()

    example_data = parse_options_data(example)
    
    
    for r in range(len(example_data)):
        d= example_data['Description'][r]
        if d =='10-Q' :
            Doc.append(example_data['Document'][r])

In [130]:
Doc

['ibm13q3_10q.htm']